# Install and Import All Libraries

In [ ]:
#install required packages

!pip install pathway bokeh --quiet

In [ ]:
# suppress specific user warnings from Bokeh

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [ ]:
# core Libraries

import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# Pathway & Bokeh

import pathway as pw
import panel as pn
import bokeh.plotting

# Load and Prepare Dataset

In [ ]:
#load dataset

df = pd.read_csv('/content/dataset.csv')
df.head()

In [ ]:
#check for missing values

df.isna().sum()

## Sort by timestamps

In [ ]:
# combine and convert date-time fields into single timestamp

df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

In [ ]:
# sort by time

df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
# compute occupancy rate

df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

In [ ]:
# export relvant columns for streaming

df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "QueueLength",
    "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)

# Define Schema and Create Real-Time Stream

In [ ]:
# define schema for Pathway stream

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: bool
    VehicleType: str

In [ ]:
# create real-time stream from CSV

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

# Feature Engineering and Preprocessing

In [ ]:
# define UDFs for converting traffic and vehicle types to numeric factors

@pw.udf
def traffic_to_level(traffic: str) -> int:
    return {"low": 0, "average": 1, "high": 2}.get(traffic, 1)

@pw.udf
def vehicle_factor(vtype: str) -> int:
    return {"cycle": 0, "bike": 1, "car": 2, "truck": 3}.get(vtype, 1)

In [ ]:
# apply transformations to stream data

fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    OccupancyRate = data.Occupancy / data.Capacity,
    TrafficLevel = traffic_to_level(data.TrafficConditionNearby),
    VehicleFactor = vehicle_factor(data.VehicleType),
    IsSpecialDay = data.IsSpecialDay,
    QueueLength = data.QueueLength,
    lot = data.SystemCodeNumber,
    instance = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d") + "_" + data.SystemCodeNumber
)

# Model 2: Demand-Based Dynamic Pricing

In [ ]:
# demand calculation function

@pw.udf
def compute_demand(occ: float, queue: float, traffic: float, special: bool, vehicle: float) -> float:
    α, β, γ, δ, ε = 1, 0.5, 0.2, 2, 0.8
    demand = α * occ + β * queue - γ * traffic + δ * int(special) + ε * vehicle
    return demand

In [ ]:
# price calculation function

@pw.udf
def compute_price(base: float, demand: float, max_demand: float = 50.0, λ: float = 0.5) -> float:
    norm_demand = min(demand / max_demand, 1.0)
    raw_price = base * (1 + λ * norm_demand)
    return max(0.5 * base, min(raw_price, 2.0 * base))

In [ ]:
# daily rolling window aggregation

import datetime
reduced = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.instance,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.OccupancyRate),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_sum = pw.reducers.sum(pw.this.TrafficLevel),
        vehicle_sum = pw.reducers.sum(pw.this.VehicleFactor),
        special_any = pw.reducers.max(pw.this.IsSpecialDay),
        n = pw.reducers.count(),
        lot = pw.reducers.min(pw.this.lot)
    )
    .with_columns(
        occ_avg = pw.this.occ_sum / pw.this.n,
        queue_avg = pw.this.queue_sum / pw.this.n,
        traffic_avg = pw.this.traffic_sum / pw.this.n,
        vehicle_avg = pw.this.vehicle_sum / pw.this.n
    )
)

In [ ]:
# compute demand

delta_window = reduced.with_columns(
    demand = compute_demand(
        reduced.occ_avg,
        reduced.queue_avg,
        reduced.traffic_avg,
        reduced.special_any,
        reduced.vehicle_avg
    )
)

In [ ]:
# compute price using previously created demand

delta_window = delta_window.with_columns(
    price = compute_price(10.0, delta_window.demand)
)

# Live Plot: Daily Dynamic Pricing by Lot

In [ ]:
pn.extension()

# define interactive Bokeh plot

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: MODEL 2 - Daily Parking Price",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=6, color="green", marker="circle")

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    return fig

# launch panel dashboard
viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()
viz.servable()

# Export Model Outputs to JSONL and Run Pipeline

In [ ]:
# save output to JSONL for further analysis

pw.io.jsonlines.write(
    delta_window.select(pw.this.t, pw.this.lot, pw.this.price),
    "pricing_output.jsonl"
)

# run the Pathway computation
pw.run()

# Load Output and Generate Interactive Plot (Full Timeline)

In [ ]:
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10_10 as palette
import pandas as pd

output_notebook()

#load and prepare data
df = pd.read_json("pricing_output.jsonl", lines=True)
df["t"] = pd.to_datetime(df["t"])                       # Safe datetime
df["price"] = pd.to_numeric(df["price"], downcast="float")
df["lot"] = df["lot"].astype(str)

#define Bokeh figure
fig = figure(
    height=500,
    width=900,
    title="Daily Pricing Across Parking Lots",
    x_axis_type="datetime",
    tools="pan,wheel_zoom,box_zoom,reset"
)

#add hover tool
hover = HoverTool(
    tooltips=[
        ("Lot", "@lot"),
        ("Time", "@t{%F %H:%M}"),
        ("Price", "@price{$0.00}")
    ],
    formatters={"@t": "datetime", "@price": "printf"},
    mode='vline'
)
fig.add_tools(hover)

#plot each lot safely
for i, lot in enumerate(sorted(df["lot"].unique())):
    df_lot = df[df["lot"] == lot][["t", "price", "lot"]]  # ✅ Restrict to safe columns only
    source = ColumnDataSource(df_lot)

    fig.line("t", "price", source=source, legend_label=f"Lot {lot}",
             color=palette[i % len(palette)], line_width=2)

#final plot formatting
fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.legend.location = "top_left"
fig.legend.label_text_font_size = "8pt"

#show and save
show(fig)
output_file("daily_pricing_comparison.html")

# Save Plot as HTML

In [ ]:
from google.colab import files
files.download("daily_pricing_comparison.html")

# Plot Each Lot in Separate Panels

In [ ]:
from bokeh.layouts import column
from bokeh.palettes import Category10_10

plots = []
for i, lot in enumerate(sorted(df["lot"].unique())):
    df_lot = df[df["lot"] == lot].copy()

    #only include safe columns to prevent precision warnings
    source = ColumnDataSource(df_lot[["t", "price", "lot"]])

    p = figure(
        height=300,
        width=800,
        title=f"Daily Price: Lot {lot}",
        x_axis_type="datetime",
        tools="pan,wheel_zoom,box_zoom,reset"
    )

    p.line("t", "price", source=source,
           line_width=2, color=Category10_10[i % len(Category10_10)])
    p.scatter("t", "price", source=source, size=5, marker="circle", color="black")

    hover = HoverTool(
        tooltips=[("Lot", "@lot"), ("Time", "@t{%F %H:%M}"), ("Price", "@price{$0.00}")],
        formatters={"@t": "datetime", "@price": "printf"},
        mode='vline'
    )
    p.add_tools(hover)
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    plots.append(p)

#show all plots
show(column(*plots))

#save to HTML
output_file("model2_price_trends.html", title="Model 2: Daily Parking Price Trends")

In [ ]:
files.download("model2_price_trends.html")